In [1]:
from gol_apis import RosterActivitiesService
from gol_apis import RosterTagService
from gol_apis import CheckScheduleService
from gol_apis import CrewMemberInfoService
import api_tools
from api_tools.setup import setup_crew_info_dataframe
from api_tools.setup import setup_checks_dataframe
from api_tools.setup import setup_tag_dataframe
from api_tools.setup import setup_activities_dataframe

from gol_logger import Logger
import pandas as pd
import multiprocessing as mp
import requests, time
from datetime import datetime as dt
from datetime import timedelta as td
from utils import funcs
from utils.progress import step
from utils.progress import BAR
from itertools import chain

from modules.generate_data import get_data_neolude
from modules.generate_data import get_rosters
from modules.generate_data import get_crew_info
from modules.generate_data import get_checks

import sys
from config.variables import user

In [2]:
training_code = 'CRM'

In [3]:
# Dados Crew Info
crew = pd.read_csv('data/crew/crew_info.csv', sep=';', encoding='utf-8-sig')
crew.cif = crew.cif.astype(int).astype(str).str.zfill(8)
crew = crew[crew['funcao'].isin(['CMT', 'COP'])]
crew = crew[['cif', 'guerra', 'funcao', 'cargo', 'base']]
crew

,cif,guerra,funcao,cargo,base
0,00000020,BORIO,CMT,IBX,CGH
1,00000021,GRABLER,CMT,IBX,GRU
2,00000022,DANTAS,CMT,IBX,CGH
3,00000024,BORBA,CMT,NaN,GRU
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH
...,...,...,...,...,...
5479,99999913,PIAZZA,CMT,IB,GRU
5480,99999916,GALENO CABRAL,CMT,IB,GRU
5481,99999917,EDWARD,CMT,IB,GRU
5482,99999918,CESIDIO,CMT,IB,GRU


In [4]:
# Header para filtrar dados do Neolude
header_neolude = ['RegistrationNumber',
                  'UserName',
                  'BusinessUnitName',
                  'Post',
                  'CPF',
                  'Email',
                  'CourseName',
                  'CourseUserStatus',
                  'GradeFormatted',
                  'LastAccessDate',
                  'CreatedDate',
                  'CompletionDate']

In [5]:
# Dados Neolude
training = pd.read_csv(f'data/neolude/{training_code}.csv', sep=';', encoding='utf-8-sig')
training = training[header_neolude]
training['tipo'] = pd.to_numeric(training['RegistrationNumber'], errors='coerce').notnull()
training = training[training['tipo']]
training['RegistrationNumber'] = training['RegistrationNumber'].fillna(0)
training['RegistrationNumber'] = training['RegistrationNumber'].astype(int).astype(str).str.zfill(8)
training['LastAccessDate'] = pd.to_datetime(training['LastAccessDate'])
training['CreatedDate'] = pd.to_datetime(training['CreatedDate'])
training['CompletionDate'] = pd.to_datetime(training['CompletionDate'])
training.columns = ['cif', 'name', 'base', 'funcao', 'cpf', 'email', 'course_name','status', 'nota', 'ultimo_acesso', 'criacao_acesso', 'data_conclusao', 'tipo']
training = training[['cif', 'course_name','status', 'nota', 'data_conclusao']]
training = training.sort_values(by=['cif','data_conclusao'])
training = training.drop_duplicates(subset = ['cif'], keep = 'last')
training

,cif,course_name,status,nota,data_conclusao
31,00000019,CRM - Corporate Resource Management,Concluído,"0,00",2014-01-21
28,00000021,CRM - Corporate Resource Management,Concluído,"0,00",2013-07-11
24,00000022,CRM - Corporate Resource Management,Concluído,"0,00",2013-10-04
19,00000026,CRM - Corporate Resource Management,Concluído,"0,00",2014-01-22
17,00000029,CRM - Corporate Resource Management,Concluído,"0,00",2014-01-15
9,00000035,CRM - Corporate Resource Management,Concluído,"0,00",2014-01-22
16,00000044,CRM - Corporate Resource Management,Concluído,"0,00",2014-01-14
8,00000049,CRM - Corporate Resource Management,Concluído,"0,00",2014-01-14
7,00000050,CRM - Corporate Resource Management,Concluído,"0,00",2014-01-17
11,00000083,CRM - Corporate Resource Management,Concluído,"0,00",2014-01-15


In [6]:
# Dados de Rosters
rosters = funcs.generate_data('data/rosters/')
rosters.cif = rosters.cif.astype(str).str.zfill(8)

importing: 2020-01...
importing: 2020-02...
importing: 2020-03...
importing: 2020-04...
importing: 2020-05...
importing: 2020-06...
importing: 2020-07...
importing: 2020-08...
importing: 2020-09...
importing: 2020-10...
importing: 2020-11...
importing: 2020-12...
importing: 2021-01...


In [7]:
inss = rosters.copy()
inss['dep'] = pd.to_datetime(inss['dep'], format='%Y-%m-%d')
inss['dep'] = inss['dep'].dt.floor('D')
inss = inss[(inss['code'] == 'INSS') & (inss['dep'] == pd.to_datetime(dt.date(dt.now())))]
inss = inss[['cif', 'code']]
inss.columns = ['cif', 'inss']
inss

,cif,inss
65631,00001962,INSS
65632,00002329,INSS
65634,00002976,INSS
65643,00010705,INSS
65648,00004196,INSS
...,...,...
66957,00005916,INSS
66959,00006334,INSS
66976,00041650,INSS
66978,00042249,INSS


In [8]:
lnr = rosters.copy()
lnr['dep'] = pd.to_datetime(lnr['dep'], format='%Y-%m-%d')
lnr['dep'] = lnr['dep'].dt.floor('D')
lnr = lnr[(lnr['code'] == 'LIC-SREM') & (lnr['dep'] == pd.to_datetime(dt.date(dt.now())))]
lnr = lnr[['cif', 'code']]
lnr.columns = ['cif', 'lnr']
lnr

,cif,lnr
65635,00023872,LIC-SREM
65636,00023903,LIC-SREM
65638,00024141,LIC-SREM
65649,00004453,LIC-SREM
65651,00004615,LIC-SREM
...,...,...
66972,00046284,LIC-SREM
66973,00046328,LIC-SREM
66974,00046330,LIC-SREM
66975,00046333,LIC-SREM


In [9]:
rosters = rosters[rosters['code'].isin(funcs.training_filter(training_code))]
rosters = rosters.sort_values(by=['cif','dep'])
rosters = rosters[['cif', 'code', 'fr', 'dep']]
rosters = rosters.drop_duplicates(subset = ['cif'], keep = 'last')
rosters

,cif,code,fr,dep
30497,00000020,C-CRMCORP,CGH,2020-10-06 11:30:00
17060,00000026,C-CRMCORP,CGH,2020-08-04 11:30:00
19376,00000029,C-CRMCORP,CGH,2020-11-04 11:30:00
154473,00000030,C-CRMCORP,CGH,2020-02-21 11:30:00
93138,00000031,C-CRMCORP,CGH,2020-02-13 11:30:00
...,...,...,...,...
84144,00045144,C-CRMCORP,FOR,2020-03-12 11:30:00
84166,00045150,C-CRMCORP,FOR,2020-03-12 11:30:00
84169,00045166,C-CRMCORP,FOR,2020-03-12 11:30:00
84170,00045167,C-CRMCORP,FOR,2020-03-12 11:30:00


In [10]:
# Dados de Checks
checks = pd.read_csv('data/checks/check_schedule.csv', sep=';', encoding='utf-8-sig')
checks.cif = checks.cif.astype(str).str.zfill(8)
checks = checks[checks['checkTypeCode'] == training_code]
checks = checks[['cif', 'checkTypeCode', 'expiryDt']]
checks

,cif,checkTypeCode,expiryDt
83,00023228,CRM,2021-02-28
88,00008027,CRM,2021-05-31
97,00010655,CRM,2019-12-31
100,00036094,CRM,2021-04-30
111,00004559,CRM,2021-02-28
...,...,...,...
55298,00010132,CRM,2021-03-30
55306,00012768,CRM,2021-03-30
55324,00022851,CRM,2021-11-30
55326,00022849,CRM,2022-11-30


In [11]:
# Merge dos dados
data_all = pd.merge(crew, checks[['cif', 'checkTypeCode', 'expiryDt']], on='cif', how='left')
data_all = data_all.merge(training, on='cif', how='left')
data_all = data_all.merge(rosters[['cif', 'code', 'dep']], on='cif', how='left')
data_all = data_all.merge(inss, on='cif', how='left')
data_all = data_all.merge(lnr, on='cif', how='left')
data_all

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr
0,00000020,BORIO,CMT,IBX,CGH,CRM,2022-10-31,NaN,NaN,NaN,NaT,C-CRMCORP,2020-10-06 11:30:00,NaN,NaN
1,00000021,GRABLER,CMT,IBX,GRU,CRM,2021-04-30,CRM - Corporate Resource Management,Concluído,"0,00",2013-07-11,NaN,NaN,NaN,NaN
2,00000022,DANTAS,CMT,IBX,CGH,CRM,2021-10-31,CRM - Corporate Resource Management,Concluído,"0,00",2013-10-04,NaN,NaN,NaN,NaN
3,00000024,BORBA,CMT,NaN,GRU,CRM,2021-05-31,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,00000026,FERNANDO FARIAS,CMT,IBX,CGH,CRM,2022-08-31,CRM - Corporate Resource Management,Concluído,"0,00",2014-01-22,C-CRMCORP,2020-08-04 11:30:00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029,99999913,PIAZZA,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2030,99999916,GALENO CABRAL,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2031,99999917,EDWARD,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2032,99999918,CESIDIO,CMT,IB,GRU,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [12]:
# Formatação do dataFrame de Dados
validation = data_all.copy()

# Ajuste de data do vencimento
validation['expiryDt'] = pd.to_datetime(validation['expiryDt'], format='%Y-%m-%d')
validation['expiryDt'] = validation['expiryDt'].dt.floor('D')

validation['dep'] = validation['dep'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['dep'] = pd.to_datetime(validation['dep'], format='%Y-%m-%d')
validation['dep'] = validation['dep'].dt.floor('D')

validation['data_conclusao'] = validation['data_conclusao'].fillna(pd.to_datetime(dt(2035,12,31)))
validation['data_conclusao'] = pd.to_datetime(validation['data_conclusao'], format='%Y-%m-%d')
validation['data_conclusao'] = validation['data_conclusao'].dt.floor('D')

validation = validation[validation['expiryDt'] < (pd.to_datetime(dt.date(dt.now())) + pd.offsets.MonthBegin(0))]
validation['tempo_de_conclusao (dias)'] = pd.to_datetime(dt.date(dt.now())) - validation['data_conclusao']

validation['tempo_de_conclusao (dias)'] = validation['tempo_de_conclusao (dias)'].dt.days

validation = validation[validation['dep'] < (pd.to_datetime(dt.date(dt.now())))]
validation['atualizar'] = (validation['status'] == 'Concluído') & (validation['tempo_de_conclusao (dias)'] < 90)

validation

,cif,guerra,funcao,cargo,base,checkTypeCode,expiryDt,course_name,status,nota,data_conclusao,code,dep,inss,lnr,tempo_de_conclusao (dias),atualizar
63,00000509,ALMEIDA JUNIOR,CMT,NaN,SDU,CRM,2021-01-28,NaN,NaN,NaN,2035-12-31,C-CRMCORP,2021-01-05,NaN,NaN,-5467,False
66,00000601,BERGMANN,CMT,NaN,POA,CRM,2021-01-28,NaN,NaN,NaN,2035-12-31,C-CRMCORP,2021-01-05,NaN,NaN,-5467,False
275,00002574,BERTINI,CMT,IR,SDU,CRM,2021-01-28,NaN,NaN,NaN,2035-12-31,C-CRMCORP,2021-01-08,NaN,NaN,-5467,False
333,00003066,ERNESTO,CMT,IR,GIG,CRM,2021-01-28,NaN,NaN,NaN,2035-12-31,C-CRMCORP,2021-01-08,NaN,NaN,-5467,False
532,00005390,GUSTAVO CARRARA,CMT,NaN,GRU,CRM,2021-01-31,NaN,NaN,NaN,2035-12-31,C-CRMCORP,2021-01-07,NaN,NaN,-5467,False
595,00006017,FORNECK,CMT,NaN,BSB,CRM,2021-01-31,NaN,NaN,NaN,2035-12-31,C-CRMCORP,2021-01-08,NaN,NaN,-5467,False
1025,00020982,TON SOARES,COP,NaN,CGH,CRM,2021-01-28,NaN,NaN,NaN,2035-12-31,C-CRMCORP,2021-01-05,NaN,NaN,-5467,False
1174,00023924,NICOLAS PONTES,COP,NaN,CGH,CRM,2021-01-28,NaN,NaN,NaN,2035-12-31,C-CRMCORP,2021-01-05,NaN,NaN,-5467,False
1177,00023930,EVANGELISTA,COP,NaN,BSB,CRM,2021-01-31,NaN,NaN,NaN,2035-12-31,C-CRMCORP,2021-01-06,NaN,NaN,-5467,False
1519,00035318,PEDRO GUIMARAES,COP,NaN,GIG,CRM,2021-01-28,NaN,NaN,NaN,2035-12-31,C-CRMCORP,2021-01-08,NaN,NaN,-5467,False


In [13]:
validation.to_csv(f'./data/output/CRM.csv', index=False, sep=';', encoding='utf-8-sig')